## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-29-22-46-41 +0000,nypost,New York gaming board set to award up to three...,https://nypost.com/2025/11/29/us-news/new-york...
1,2025-11-29-22-42-00 +0000,wsj,Protesters Clash with Homeland Security Agents...,https://www.wsj.com/us-news/protesters-clash-w...
2,2025-11-29-22-31-05 +0000,latimes,"After Prop. 50, San Diego may lose its only Re...",https://www.latimes.com/california/story/2025-...
3,2025-11-29-22-27-56 +0000,bbc,Ukraine hits tankers in Black Sea in escalatio...,https://www.bbc.com/news/articles/cp9ky5z2gx0o...
4,2025-11-29-22-09-35 +0000,nyt,In Announcing Pardon of Drug Trafficker While ...,https://www.nytimes.com/2025/11/29/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,35
394,national,10
47,fire,10
264,president,9
5,up,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
154,2025-11-29-06-53-08 +0000,nypost,Trump says he plans to pardon former Honduran ...,https://nypost.com/2025/11/29/us-news/trump-pl...,85
42,2025-11-29-19-34-03 +0000,nypost,White House hits back after Dem blames Trump f...,https://nypost.com/2025/11/29/us-news/white-ho...,78
187,2025-11-29-00-35-54 +0000,wapo,Trump plans to pardon former Honduran presiden...,https://www.washingtonpost.com/world/2025/11/2...,75
180,2025-11-29-01-12-00 +0000,wsj,Trump to Pardon Honduran Ex-President Serving ...,https://www.wsj.com/politics/trump-to-pardon-f...,74
71,2025-11-29-16-32-00 +0000,bbc,Trump to pardon ex-Honduras president convicte...,https://www.bbc.com/news/articles/c1kpm0rvxepo...,71


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
154,85,2025-11-29-06-53-08 +0000,nypost,Trump says he plans to pardon former Honduran ...,https://nypost.com/2025/11/29/us-news/trump-pl...
194,45,2025-11-28-23-50-12 +0000,nypost,Treasury Secretary Scott Bessent moves to ‘cut...,https://nypost.com/2025/11/28/us-news/treasury...
42,43,2025-11-29-19-34-03 +0000,nypost,White House hits back after Dem blames Trump f...,https://nypost.com/2025/11/29/us-news/white-ho...
15,38,2025-11-29-21-45-09 +0000,nypost,Parents of slain Israeli hostage still held in...,https://nypost.com/2025/11/29/world-news/paren...
74,36,2025-11-29-16-26-44 +0000,nypost,Mom of NJ teen killed outside NYC ax-throwing...,https://nypost.com/2025/11/29/us-news/mom-of-n...
151,34,2025-11-29-07-14-36 +0000,bbc,US halts all asylum claim decisions after Nati...,https://www.bbc.com/news/articles/cdxweyy157go...
97,33,2025-11-29-13-50-50 +0000,nypost,Russia attack on Kyiv kills three as efforts t...,https://nypost.com/2025/11/29/world-news/russi...
35,30,2025-11-29-20-03-34 +0000,nypost,Pope Leo visits Turkey’s famous Blue Mosque in...,https://nypost.com/2025/11/29/world-news/pope-...
70,28,2025-11-29-16-36-40 +0000,bbc,Sultana claims new Corbyn party carrying out '...,https://www.bbc.com/news/articles/cpq4l0rn3y1o...
6,27,2025-11-29-22-02-09 +0000,nypost,Israel considers ramping up airstrikes on Syri...,https://nypost.com/2025/11/29/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
